# calculates the seasonal anomalies and the seasonal percentile categories from the gridded VCSN monthly files, using the NZ 6 regions shapefiles and the [salem](https://salem.readthedocs.io/en/stable/) library for spatial extraction

In [9]:
# Paramaters 

var_name = 'Rain_bc'
# var_name = 'Tmin_N'
# var_name = 'Tmax_N'
# var_name = 'Tmean_N'
# var_name = 'SoilM'
# var_name = 'Wind'
# var_name = 'Rad'

in ['Agent', 'Lat', 'Longt', 'Date', 'MSLP', 'PET', 'Rain', 'RH', 'SoilM',
       'ETmp', 'Rad', 'TMax', 'Tmin', 'VP', 'Wind', 'Rain_bc', 'Tmax_N',
       'Tmin_N']

In [10]:
import os
import sys
import pathlib

In [11]:
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
from itertools import product

In [12]:
import salem
import geopandas as gpd

In [14]:
salem.__version__

'0.3.0-1-g697762b'

In [5]:
import xarray as xr

### function to calculate the anomalies with respect to the 1981 - 2010 climatology 

In [6]:
def demean(x): 
    return x - x.loc['1981':'2010',].mean()

In [7]:
var_name

'Rain_bc'

### big_var is the simplified version of the variable, so Rain_bc --> RAIN

In [9]:
big_var = var_name.split('_')[0].upper()

In [10]:
HOME = pathlib.Path.home()

In [11]:
dpath = HOME / 'operational/VCSN/data/NC/MONTHLY/' / var_name.upper()

In [12]:
var_name.upper()

'TMEAN_N'

In [13]:
dpath

PosixPath('/home/nicolasf/operational/VCSN/data/NC/MONTHLY/TMEAN_N')

In [14]:
dset = salem.open_xr_dataset(dpath / f'VCSN_gridded_{var_name}_1979-01_2019-12.nc') 

In [15]:
dset

<xarray.Dataset>
Dimensions:  (lat: 257, lon: 241, time: 492)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-31 1979-02-28 ... 2019-12-31
  * lon      (lon) float64 166.5 166.5 166.6 166.6 ... 178.3 178.4 178.4 178.5
  * lat      (lat) float64 -47.27 -47.23 -47.17 -47.12 ... -34.52 -34.48 -34.42
Data variables:
    agent    (lat, lon) float32 ...
    Tmean_N  (time, lat, lon) float64 ...
Attributes:
    pyproj_srs:  +proj=longlat +datum=WGS84 +no_defs

### calculates the seasonal average (or sum if Rain_bc is the variable )

In [16]:
if var_name == 'Rain_bc': 
    dset = dset.rolling(time=3, min_periods=3).sum()
else: 
    dset = dset.rolling(time=3, min_periods=3).mean()

/home/nicolasf/anaconda3/envs/salem/lib/python3.8/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [17]:
dset = dset.isel(time=slice(2,None))

In [18]:
nz_regions = gpd.read_file(HOME / 'research' / 'Smart_Ideas' / 'data' / 'shapefiles' / 'NZ_regions' / 'NZ_6_regions' / 'NZ_regions_corrected.shp') 

In [19]:
nz_regions

,OBJECTID,Id,gridcode,Shape_Leng,Shape_Area,Location,geometry
0,1,1,1,85.215338,5.032753,NNI,"MULTIPOLYGON (((174.70530 -38.17377, 174.70545..."
1,2,2,2,12.336015,2.994028,WNI,"MULTIPOLYGON (((175.13516 -41.37745, 175.13507..."
2,3,3,3,14.235493,3.775388,ENI,"MULTIPOLYGON (((175.85595 -41.35970, 175.85595..."
3,4,4,4,34.656463,3.066280,NSI,"MULTIPOLYGON (((171.32620 -42.12355, 171.32602..."
4,5,5,6,20.191504,4.827228,ESI,"MULTIPOLYGON (((170.21675 -46.05955, 170.21609..."
5,6,6,5,42.941379,9.057410,WSI,"MULTIPOLYGON (((169.20749 -46.66371, 169.20742..."


#### checks that the crs is correct: should be epsg 4272 

In [20]:
nz_regions.crs

<Geographic 2D CRS: EPSG:4272>
Name: NZGD49
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: New Zealand - onshore and nearshore
- bounds: (165.87, -47.65, 179.27, -33.89)
Datum: New Zealand Geodetic Datum 1949
- Ellipsoid: International 1924
- Prime Meridian: Greenwich

In [32]:
opath_root = HOME / 'research' / 'Smart_Ideas' / 'outputs' / 'targets' / 'NZ_regions' / 'NZ_6_regions'

In [33]:
if not opath_root.exists(): 
    opath_root.mkdir(parents=True)

### defines the number of quantiles we want 

In [34]:
num_quantiles = 3

In [35]:
quant_values = np.linspace(0, 1, num_quantiles + 1, endpoint=True)

In [36]:
quant_values = quant_values[1:-1]

In [37]:
quant_values

array([0.33333333, 0.66666667])

In [38]:
col_labs = [f"Q{int(x)}" for x in (quant_values*100)]

In [39]:
col_labs

['Q33', 'Q66']

In [40]:
# f, axes = plt.subplots(nrows=3, ncols=2)
# axes = axes.flatten()

quantiles_dict = {}

for i, region_name in enumerate(['NNI','ENI','WNI','NSI','WSI','ESI']): 
    
    shape = nz_regions.query(f"Location == '{region_name}'")
    
    region = dset.salem.subset(shape=shape)

    region = region.salem.roi(shape=shape, all_touched=True)
        
    ts = region.mean(dim=['lat','lon'])
    
    ts_df = ts[var_name].to_dataframe()
            
    ts_series = ts_df.loc[:,var_name]
    
    ts_series_cat = []
    
    quantiles_list = []
    
    for month in range(1, 13):
        
        ts_series_m = ts_series[ts_series.index.month == month]
        
        clim = ts_series_m.loc['1981':'2010']
        
        quantiles = [clim.quantile(q=q) for q in quant_values.tolist()]
        
        quantiles_list.append(quantiles.copy())
        
        quantiles.insert(0, -np.inf)
        
        quantiles.append(np.inf)
        
        ts_series_m_cats = pd.cut(ts_series_m, quantiles, labels=list(range(1, num_quantiles + 1)))
        
        ts_series_cat.append(ts_series_m_cats)
        
        del(quantiles)
     
    quantiles_dict[region_name]  = np.array(quantiles_list)
    
    ts_series_cat = pd.concat(ts_series_cat, axis=0)
    
    ts_series_cat = ts_series_cat.sort_index()
    
    ts_df.loc[:,f'cat_{num_quantiles}'] = ts_series_cat
    
    ts_df.loc[:,'anomalies'] = ts_df.loc[:,var_name].groupby(ts_df.index.month).apply(demean)
    
    opath = opath_root / big_var / region_name 
    
    if not opath.exists(): 
        opath.mkdir(parents=True)
        
    ts_df.to_csv(opath / f'TS_NZ_region_{region_name}_{big_var}_{num_quantiles}_quantiles_anoms_salem.csv')
    
    #descriptive statistics per quantile category 
    
    ts_df.groupby(ts_df.loc[:,f'cat_{num_quantiles}']).describe().to_csv(opath / f'descriptive_stats_{region_name}__{big_var}_{num_quantiles}_salem.csv')
    
    print(f"region {region_name} processed for variable {big_var}")
    

region NNI processed for variable TMEAN
region ENI processed for variable TMEAN
region WNI processed for variable TMEAN
region NSI processed for variable TMEAN
region WSI processed for variable TMEAN
region ESI processed for variable TMEAN


### saves the climatological terciles calculated from the VCSN regional aggregates

In [41]:
quantiles_list = []
for region_name in ['NNI','ENI','WNI','NSI','WSI','ESI']: 
    df = pd.DataFrame(quantiles_dict[region_name])
    df.index = range(1, 13)
    df.index.name = 'season'
    df.columns = pd.MultiIndex.from_product([[region_name],col_labs])
    quantiles_list.append(df)

In [42]:
quantiles_df = pd.concat(quantiles_list, axis=1)

In [43]:
quantiles_df

NNI                   ENI                   WNI             \
              Q33        Q66        Q33        Q66        Q33        Q66   
season                                                                     
1       16.162322  16.689360  14.631518  15.323342  14.097799  14.820892   
2       17.521588  18.117734  15.967439  16.575433  15.619348  16.200577   
3       17.543217  18.029687  15.840041  16.220799  15.465456  15.944101   
4       16.276928  16.831057  14.270189  14.868288  13.920626  14.542433   
5       14.196003  14.756303  12.062712  12.718081  11.791206  12.487227   
6       11.929649  12.393454   9.945910  10.234096   9.637784  10.096512   
7       10.060553  10.493580   8.005132   8.581372   7.764019   8.375355   
8        9.327370   9.840370   7.295052   7.780403   7.101377   7.663780   
9        9.833784  10.315089   7.978668   8.339665   7.753814   8.197758   
10      11.113474  11.473610   9.202016   9.760828   9.002470   9.507164   
11      12.663317  13.082596  11.019657  11.386040  10.598933  11.088245   
12      14.445608  14.937938  12.845351  13.513533  12.331196  13.126211   

              NSI                   WSI                   ESI             
              Q33        Q66        Q33        Q66        Q33        Q66  
season                                                                    
1       12.442040  13.008797  10.918563  11.429266  12.625489  13.456594  
2       13.640947  14.359592  12.099764  12.683038  13.922874  14.522162  
3       13.648228  14.116516  11.985550  12.622185  13.657104  14.283504  
4       12.266969  12.714772  10.430209  11.089917  11.887382  12.556928  
5        9.935456  10.462286   8.160988   8.625773   9.401919  10.016357  
6        7.375768   7.799300   5.578766   6.029423   6.683452   7.273791  
7        5.281877   5.989573   3.453042   3.987287   4.541899   5.167835  
8        4.557124   5.278942   2.814963   3.465440   4.067740   4.340551  
9        5.479288   6.006328   3.759792   4.376392   5.087315   5.556088  
10       7.062311   7.438129   5.638929   5.864831   7.008489   7.510551  
11       8.742039   9.058670   7.262977   7.762150   8.977433   9.473067  
12      10.557275  11.153107   9.105084   9.805416  10.823604  11.443097

In [36]:
quantiles_df.to_csv(opath.parent / f'Climatological_quantiles_{num_quantiles}_cat_{big_var}_salem.csv')